In [1]:
 from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
cd /content/drive/My Drive/Data Glacier/week6

/content/drive/My Drive/Data Glacier/week6


In [3]:
#Import librairies
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re

In [7]:
#!unzip Age-sex-by-ethnic-group-grouped-total-responses-census-usually-resident-population-counts-2006-2013-2018-Censuses-RC-TA-SA2-DHB.zip

Archive:  Age-sex-by-ethnic-group-grouped-total-responses-census-usually-resident-population-counts-2006-2013-2018-Censuses-RC-TA-SA2-DHB.zip
  inflating: Data8277.csv            
  inflating: DimenLookupAge8277.csv  
  inflating: DimenLookupArea8277.csv  
  inflating: DimenLookupEthnic8277.csv  
  inflating: DimenLookupSex8277.csv  
  inflating: DimenLookupYear8277.csv  


In [4]:
#read the files
#normal approach
df = pd.read_csv("Data8277.csv",delimiter=',')
df.head()


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Year,Age,Ethnic,Sex,Area,count
0,2018,0,1,1,1,795
1,2018,0,1,1,2,5067
2,2018,0,1,1,3,2229
3,2018,0,1,1,4,1356
4,2018,0,1,1,5,180


In [6]:
!pip install modin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 956 kB 43.1 MB/s 
     |████████████████████████████████| 956 kB 77.3 MB/s 
     |████████████████████████████████| 761 kB 66.5 MB/s 


In [7]:
# read the file using Dask
import dask.dataframe as dd
import modin.pandas as mpd
df1 = dd.read_csv('Data8277.csv')

df1

,Year,Age,Ethnic,Sex,Area,count
npartitions=13,,,,,,
,int64,int64,int64,int64,int64,int64
,...,...,...,...,...,...
...,...,...,...,...,...,...
,...,...,...,...,...,...
,...,...,...,...,...,...


In [8]:
#read the file using modin
df2 = mpd.read_csv('Data8277.csv')
df2

To remove this warning, run the following python code before doing dataframe operations:

    from distributed import Client

    client = Client()



,Year,Age,Ethnic,Sex,Area,count
0,2018,0,1,1,1,795
1,2018,0,1,1,2,5067
2,2018,0,1,1,3,2229
3,2018,0,1,1,4,1356
4,2018,0,1,1,5,180
...,...,...,...,...,...,...
34959667,2006,888,69,9,DHB19,34.3
34959668,2006,888,69,9,DHB22,24.5
34959669,2006,888,69,9,DHB99,..C
34959670,2006,888,69,9,DHB9999,28.8


### Note:
- I notices that reading the file in normal way (pandas) is more efficient than others methods such as Dask and Modin.
- Modin is taking more time than Dask and pandas while Dask is taking more time than pandas as well

In [18]:
#basic evauation on data columns
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re


################
# File Reading #
################

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Writing testutility.py


## Writing YAML file

In [19]:
%%writefile file.yaml
file_type: csv
dataset_name: datafile
file_name: Data8277
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - year
    - age
    - sex
    - ethnic

Writing file.yaml


In [9]:
# Read config file
import testutility as util
config_data = util.read_config_file("file.yaml")

In [10]:
config_data['inbound_delimiter']

','

In [11]:
#inspecting data of config file
config_data

{'file_type': 'csv',
 'dataset_name': 'datafile',
 'file_name': 'Data8277',
 'table_name': 'edsurv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['year', 'age', 'sex', 'ethnic']}

In [12]:
# read the file using config file
file_type = config_data['file_type']
source_file = "./" + config_data['file_name'] + f'.{file_type}'
df_sample = pd.read_csv(source_file,config_data['inbound_delimiter'])
df_sample.head()

,Year,Age,Ethnic,Sex,Area,count
0,2018,0,1,1,1,795
1,2018,0,1,1,2,5067
2,2018,0,1,1,3,2229
3,2018,0,1,1,4,1356
4,2018,0,1,1,5,180


In [13]:
#validate the header of the file
util.col_header_val(df_sample,config_data)

column name and column length validation failed
Following File columns are not in the YAML file ['area', 'count']
Following YAML columns are not in the file uploaded []


0

In [14]:
print("columns of files are:" ,df_sample.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['year', 'age', 'ethnic', 'sex', 'area', 'count'], dtype='object')
columns of YAML are: ['year', 'age', 'sex', 'ethnic']


In [15]:
if util.col_header_val(df_sample,config_data)==0:
    print("validation failed")
    # write code to reject the file
else:
    print("col validation passed")
    # write the code to perform further action
    # in the pipleine

column name and column length validation failed
Following File columns are not in the YAML file ['area', 'count']
Following YAML columns are not in the file uploaded []
validation failed


In [17]:
# write a pandas dataframe to gzipped CSV file
df_sample.to_csv("Data.csv.gz", 
           index=False, 
           compression="gzip", sep='|')